In [1]:
import requests
from requests_html import HTMLSession

In [22]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',

}

def run(username):
    stars_url = "https://github.com/" + username + "?tab=stars"
    
    session = HTMLSession()
    resp = session.get(stars_url, headers=headers)
    
    # 获取类型
    types = resp.html.xpath('//ul[@class="filter-list"]/li/a/text()')
    types_result = [k.replace("  ","") for k in [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in types]]]
    types_result = types_result[4:] # 去掉多余
#     print(types_result)
    
    # 获取类型链接
    urls = resp.html.xpath('//ul[@class="filter-list"]/li/a/@href')
    urls_result = urls[4:] # 去掉多余
#     print(urls_result)
    
    github_url = "https://github.com"
    # 每个类型的页面
    for url in urls_result:
        resp = session.get(url, headers=headers)
#         pages_up_down = resp.html.xpath('//div[@class="BtnGroup"]/a/@hrerf')
        title = resp.html.xpath('//div[@class="d-inline-block mb-1"]/h3/a/@href')
        title_url = [github_url + i for i in title]
#         print(title)
#         print(title_url)
        text = resp.html.xpath('//div[@class="py-1"]/p/text()')
        text = [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in text]]
#         print(text)
    
    # 获取项目名称、评论、stars、fork
    # 判断是否有下一页，获取链接
    
def makeMarkdown():
    details_head_contents = "<details><summary>Contents</summary>"
    details_head_List = "<details><summary>List</summary>"
    datails_font = "</details>"
    
    # 模板制作：显示样式，是否需要stars&fork
    # 支持输入文件名，可有可无".md"
    
if __name__ == '__main__':
#     username = input("input your github username: ")
    username = "anlzou"
    data = run(username)

['Productive & portable programming language for high-performance, sparse & differentiable computing']
['A design system for building faithful recreations of old UIs']
['CSS3奇思妙想，单标签实现各类图形', 'My websites']
['\n', ' 技术面试必备基础知识、Leetcode、计算机操作系统、计算机网络、系统设计、Java、Python、C++', '\n', ' 互联网 Java 工程师进阶知识完全扫盲：涵盖高并发、分布式、高可用、微服务、海量数据处理等领域知识，后端同学必看，前端同学也可学习', '\n', ' GitHub中文排行榜，帮助你发现高分优秀中文项目、更高效地吸收国人的优秀经验成果；榜单每周更新一次，敬请关注！', '\n', ' An excellent open source blog publishing application. | 一个优秀的开源博客发布应用。', '白卷是一款使用 Vue+Spring Boot 开发的前后端分离项目，附带全套开发教程。（A simple CMS developed by Spring Boot and Vue.js with development tutorials）', 'web-flash -- Admin Framework and Mobile Website Based on Spring Boot and Vue.js', '一点知识学院 是一个在线课程资料的管理系统（是对app手机端的支撑），项目框架采用SpringBoot + JPA + mysql + easyui开发。']
['A powerful, interactive charting and visualization library for browser', 'The next open source file uploader for web browsers ', '\n', 'A simple HTML5, YouTube and Vimeo player', 'Modern & flexible browser finger